# Assignment 4 - Optimization Methods
## Matteo Ghilardini
---

### Part 2: programming problem

#### Setup the environment:

Install the required libraries in the jupiter notebook environment:

In [30]:
pip install numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


Import all the required libraries

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial, pow

#### **Problem 1** (Line search)
We consider the function
$$
f(x)=x_1^2+2x_2^2
$$

##### 1. Define a function that computes $f(x)$ for any $x = (x_1, x_2)$.

In [32]:
def f(x):
    x1, x2 = x
    return x1**2 + 2*x2**2

##### 2. Define a function that computes the gradient of $f$, $\nabla f(x)$ for any $x = (x_1, x_2)$.

In [33]:
def grad_f(x):
    x1, x2 = x
    partial_x1 = 2*x1
    partial_x2 = 4*x2
    return np.array([partial_x1, partial_x2])

##### 3. Evaluate $f$ and its gradient function at $x^{(0)} = (9, 1)^T$.

In [34]:
x_0 = np.array([9.0, 1.0])

print("Function value at x_0 = ",x_0,":", f(x_0))
print("Gradient of f at x_0 = ",x_0,":", grad_f(x_0))

Function value at x_0 =  [9. 1.] : 83.0
Gradient of f at x_0 =  [9. 1.] : [18.  4.]


##### 4. Define a function `wolfe_conditions` that verifies if the first and second Wolfe conditions are verified for some $\alpha > 0$, given a function, its gradient function, a current point $x$, a direction $d$ and the parameters $\eta$ and $\bar{\eta}$ of the Wolfe conditions. 
This function should take as input
- a function $f$ (callable object)
- its gradient function $\nabla f$ (callable object)
- a current points $x$
- a direction $d$
- a step size $\alpha$
- parameters $\eta$ and $\bar{\eta}$.

and should return a tuple of 2 booleans, indicating if each of the Wolfe conditions is verified.

First wolfe: $g(\alpha^{(k)}) = f(x^{(k)} + \alpha^{(k)} d^{(k)}) \leq f(x^{(k)}) + \eta \alpha^{(k)} \nabla f(x^{(k)})^T d^{(k)} = \ell(\alpha^{(k)})$

Second wolfe: $\nabla f(x^{(k)} + \alpha^{(k)} d^{(k)})^T d^{(k)} \geq \bar{\eta} \nabla f(x^{(k)})^T d^{(k)}$


In [35]:
def verify_first_wolfe(f, x, alpha, grad_f, d_N, eta = 0.2):
    lhs = f(x + alpha * d_N)
    rhs = f(x) + alpha * eta * grad_f(x).dot(d_N)
    return lhs <= rhs

def verify_second_wolfe(f, x, alpha, grad_f, d_N, eta_bar = 0.8):
    lhs = grad_f(x + alpha * d_N).dot(d_N)
    rhs = eta_bar * grad_f(x).dot(d_N)
    return lhs >= rhs

def wolfe_conditions(f, grad_f, x, d, alpha=0.1, eta=0.01, eta_bar=0.8):
    return verify_first_wolfe(f, x, alpha, grad_f, d, eta), verify_second_wolfe(f, x, alpha, grad_f, d, eta_bar)

##### 5. Test the `wolfe_conditions` function for the function $f$ at $x^{(0)} = (9, 1)^T$ with $d^{(0)} = −\nabla f(x^{(0)})$, $\alpha = 0.05$, $\eta = 0.01$, $\bar{\eta} = 0.8$ 

In [36]:
# Compute the direction d^(0)
d_0 = -grad_f(x_0)

# Test the wolfe_conditions function
alpha = 0.05
eta = 0.01
eta_bar = 0.8

verify_first, verify_second = wolfe_conditions(f, grad_f, x_0, d_0, alpha, eta, eta_bar)

print("First Wolfe condition satisfied:", verify_first)
print("Second Wolfe condition satisfied:", verify_second)

First Wolfe condition satisfied: True
Second Wolfe condition satisfied: False


##### 6. Define a function `backtracking` that implements the backtracking line search algorithm. 
This function should take as input
- a function $f$ (callable object)
- its gradient function $\nabla f$ (callable object)
- a current points $x$
- a direction $d$
- a maximum step size $\bar{\alpha}$
- parameter $\eta$ of the first Wolfe conditions
and should return the found value of $\alpha ^*$ and the new iterate $x_{new} = x + \alpha ^* d$. It should also call the function `wolfe_conditions`.

In [44]:
def backtracking(f, grad_f, x, d, alpha_max, eta):
    alpha = alpha_max
    while not all(wolfe_conditions(f, grad_f, x, d, alpha, eta, eta_bar)): # Repeat until both Wolfe conditions are satisfied
        alpha *= 0.5  # Reduce the step size (i.e. alpha)
    x_new = x + alpha * d
    return alpha, x_new

##### 7. Test the `backtracking` function for the function $f$ at $x^(0) = (9, 1)^T$ with $d^{(0)} =−\nabla f(x^{(0)})$, $\bar{\alpha} = 10$ and $\eta = 0.01$.

In [45]:
x_0 = np.array([9.0, 1.0])
d_0 = -grad_f(x_0)
alpha_max = 10
eta = 0.01

alpha_star, x_new = backtracking(f, grad_f, x_0, d_0, alpha_max, eta)

print("alpha*:", alpha_star)
print("x_new:", x_new)

alpha*: 0.625
x_new: [-2.25 -1.5 ]


#### **Problem 2** (Unconstrained optimisation)

Consider the function $f(x) = x_1^3 − x_1 + x_2^3 − x_2$ and solve the following problem
$$
\min_{x \in \R^2} f(x),
$$

using your own implementation of each of the following algorithms:
- Gradient descent with a fixed constant step size, chosen by you.
- Gradient descent with backtracking line search.
- Newton’s method with backtracking line search.

with starting point $x^{(0)} = (1, 1)^T$.

#### **Problem 3** (Finite difference)

##### 1. Define a function `forward_finite_difference` that compute the forward finite difference approximation of the first derivative of a function $f$ at a point $x$. The function `forward_finite_difference` must take as input a function $f$, a point $x$ and a level $t$ used to compute the finite difference.

##### 2. Define a function $f$ that compute the $sin$ of any real number $x$. Then define another function that outputs the (analytical) derivative of $f$.

##### 3. Compute the estimate of $f'(x)$ at $x = 1$ using the function `forward_finite_difference` for $t = 10^{−16}, 10^{−15} , \dots , 10^{−2}, 10^{−1}$ and compute the approximation error for each $t. Plot this error versus $t$ in _log-log scale_.

##### 4. Define a function `central_difference` that compute the central difference approximation of the first derivative of a function $f$ at a point $x$. The function `central_difference` must take as input a function $f$, a point $x$ and a level $t$ used to compute the finite difference.

##### 5. Compute the estimate of $f'(x)$ at x = 1 using the function `central_difference` for $t = 10^{−16}, 10^{−15} , \dots , 10^{−2}, 10^{−1}$ and compute the approximation error for each $t$. Plot this error versus $t$ in _log-log scale_ and on the same plot, the error of the forward finite difference scheme. Comment the results.